<a href="https://colab.research.google.com/github/shriyan44/Lab-6-DATA-301/blob/main/Copy_of_DATA_301_Lab_6A_Shriya_Nimmagadda.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CityBikes Rest API

This section asks you to fetch JSON data from the [CityBikes REST API](https://api.citybik.es/v2/) to answer some questions about bike share programs.

Please be mindful not to send requests to the server more often than necessary. In particular, make sure you separate code that requests data from the server into a separate cell so that you do not have to repeatedly request data from the server. You will lose points if your requests are not separated into their own cell.

# Question 1

Find all cities with bike share programs in the United States (country code "US"), along with their network ID (for example, "ford-gobike"). How many cities in the U.S. have bike share programs (at least that are in this API)?

In [1]:
import requests
response = requests.get("https://api.citybik.es/v2/networks?fields=id,name,location")


In [2]:
import pandas as pd

data_bikes = response.json()

df_bikes = pd.json_normalize(data_bikes, "networks")


In [3]:
df_bikesUS = df_bikes[df_bikes["location.country"] == "US"]
df_bikesUS

,id,name,location.city,location.country,location.latitude,location.longitude
75,we-cycle,WE-cycle,"Aspen, CO",US,39.194951,-106.837002
131,arborbike,ArborBike,"Ann Arbor, MI",US,42.278530,-83.745360
134,austin,Austin B-cycle,"Austin, TX",US,30.264080,-97.743550
135,bike-chattanooga,Bike Chattanooga,"Chattanooga, TN",US,35.045630,-85.309680
138,biketown,BIKETOWN,"Portland, OR",US,45.521754,-122.681079
...,...,...,...,...,...,...
562,jerseybike-guttenberg,JerseyBike,Guttenberg,US,40.792000,-74.003500
563,jerseybike-west-new-york,JerseyBike,West New York,US,40.788200,-74.014200
570,jerseybike-north-bergen,JerseyBike,North Bergen,US,40.804500,-74.013500
592,jerseybike-liberty-state-park,JerseyBike,Liberty State Park,US,40.705100,-74.052000


There are 64 bike share programs in the US

# Question 2

Construct a `DataFrame` containing data about all bike stations in all networks in the United States. Save this `DataFrame` to disk using `.to_csv()`. (You will need it in Part B of this lab.)

Use this `DataFrame` to determine the total number of bicycles in bike share programs across the United States? You may assume that the number of bikes at a station is the number of empty spaces, plus the number of available bikes.

In [4]:
import time 
stations = []
for id in df_bikesUS["id"]:
  response = requests.get("https://api.citybik.es/v2/networks/" + id)
  stations.append(response.json()["network"])
  time.sleep(0.5)
df_stations = pd.json_normalize(stations, "stations", meta=["name", "id"], meta_prefix="network", errors="ignore")
df_stations


,empty_slots,free_bikes,id,latitude,longitude,name,timestamp,extra.address,extra.last_updated,extra.renting,extra.returning,extra.uid,extra.ebikes,extra.has_ebikes,extra.bike_uids,extra.number,extra.slots,networkname,networkid
0,5.0,6,041409c38a1cc57e8a0c49b5ef8458fe,39.394798,-107.095600,Sopris View,2021-11-02T14:43:14.673000Z,Basalt,1.635864e+09,1.0,1.0,1421,NaN,NaN,NaN,NaN,NaN,WE-cycle,we-cycle
1,6.0,3,de99fa376a6ff38db656affe33d40e52,39.380066,-107.081968,Evans Rd,2021-11-02T14:43:14.732000Z,Basalt,1.635864e+09,1.0,1.0,1482,NaN,NaN,NaN,NaN,NaN,WE-cycle,we-cycle
2,6.0,1,d0a1eda25363664537ff8efb3840a740,39.396610,-107.092131,English in Action,2021-11-02T14:43:14.729000Z,Basalt,1.635864e+09,1.0,1.0,1473,NaN,NaN,NaN,NaN,NaN,WE-cycle,we-cycle
3,6.0,1,d4e64b89339be8aafa37203a16957568,39.399257,-107.090126,JW DR | El Jebel Road,2021-11-02T14:43:14.731000Z,Basalt,1.635864e+09,1.0,1.0,1474,NaN,NaN,NaN,NaN,NaN,WE-cycle,we-cycle
4,7.0,0,ee081bd0c783181e3959e907c77d7dfa,39.391593,-107.087100,East Valley Road,2021-11-02T14:43:14.734000Z,Basalt,1.635864e+09,1.0,1.0,1436,NaN,NaN,NaN,NaN,NaN,WE-cycle,we-cycle
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6707,18.0,5,df9c9c79952699cc041118a331ea9cc2,37.724893,-122.443238,Balboa Park (San Jose Ave at Sgt. John V. Youn...,2022-02-25T22:07:55.424000Z,None,1.645826e+09,1.0,1.0,501,4.0,True,NaN,NaN,NaN,Bay Wheels,bay-wheels
6708,17.0,10,5dc0d98ea835b4c9743d4e2cdd515dde,37.726266,-122.479462,SFSU University Park North - West,2022-02-25T22:07:55.739000Z,None,1.645808e+09,1.0,1.0,566,0.0,True,NaN,NaN,NaN,Bay Wheels,bay-wheels
6709,9.0,6,97871d03b345e3aa1de1dbed3640b700,37.725603,-122.476561,SFSU University Park North - East,2022-02-25T22:07:55.743000Z,None,1.645823e+09,1.0,1.0,567,1.0,True,NaN,NaN,NaN,Bay Wheels,bay-wheels
6710,6.0,13,e427fa5d07c952f21297dbd9c7693407,37.723426,-122.455102,Brighton Ave at Ocean Ave,2022-02-25T22:07:55.738000Z,None,1.645818e+09,1.0,1.0,565,1.0,True,NaN,NaN,NaN,Bay Wheels,bay-wheels


In [5]:
df_stations["empty_slots"].sum() + df_stations["free_bikes"].sum()

130942.0

Total number of bikes in US: 130942

In [6]:
from google.colab import drive

drive.mount('/content/drive')
path = '/content/drive/My Drive/stations.csv'

with open(path, 'w') as f:
  df_stations.to_csv(f)

Mounted at /content/drive


# Question 3

You have just finished touring Coit Tower in San Francisco, which is located at latitude 37.802747 and longitude -122.405861. Using your `DataFrame` from Question 2, find the nearest bike station with an available bike, based on taxicab distance (a.k.a. Manhattan distance).

_Hint:_ You can check your answer using Google Maps!

In [8]:
mindist = 1000
minidx = 100
for i in range(len(df_stations)):
  dist = abs(37.802747 - df_stations["latitude"][i]) + abs(-122.405861 - df_stations["longitude"][i])
  if dist < mindist:
    mindist = dist
    minidx = i
print(mindist)
print(minidx)

0.00465000000000515
6264


In [9]:
df_stations.loc[6265]


empty_slots                                       15.0
free_bikes                                           8
id                    eb9c95aaf839b9c7e8b18e68288862a3
latitude                                     37.771762
longitude                                  -122.398438
name                               Berry St at King St
timestamp                  2022-02-25T22:07:54.635000Z
extra.address                                     None
extra.last_updated                        1645826477.0
extra.renting                                      1.0
extra.returning                                    1.0
extra.uid                                           91
extra.ebikes                                       1.0
extra.has_ebikes                                  True
extra.bike_uids                                    NaN
extra.number                                       NaN
extra.slots                                        NaN
networkname                                 Bay Wheels
networkid 

The closest station is named The Embarcadero at Sansome St and has id: 28754859653e0a656ba53e1fa0ea21db

## Submission Instructions

- Copy this notebook to your own Drive, if you have not already.
- Restart this notebook and run the cells from beginning to end. 
  - Go to Runtime > Restart and Run All.
- Rename this notebook by clicking on "DATA 301 Lab 6A - YOUR NAMES HERE" at the very top of this page. Replace "YOUR NAMES HERE" with the first and last names of you (and your partners, for Phase 2).
- Get the link to your notebook:
  - Click on "Share" at the top-right. 
  - Change the settings to "Anyone with the link can view". 
  - Copy the sharing link into Canvas.